# 🎯 MOUAADNET-ULTRA Detection Training
## Production-Grade AI for Human Detection

**Author:** MOUAAD IDOUFKIR

### ⚡ Features:
- **Speed**: Mixed precision (FP16), efficient data loading
- **Memory**: Gradient checkpointing, optimized batch processing
- **Robust**: Full error handling, checkpoint recovery
- **Export**: ONNX with dynamic axes

---

In [ ]:
#@title 1️⃣ Setup Environment
!nvidia-smi
!pip install -q torch torchvision tqdm pycocotools opencv-python onnx

In [ ]:
#@title 2️⃣ Configuration
import os

# Auto-detect platform
if os.path.exists('/teamspace'):
    WORK = '/teamspace/studios/this_studio'
else:
    WORK = '/content'

REPO = f'{WORK}/MouaadNet-Ultra'
DATA = f'{WORK}/coco'
CKPT = f'{WORK}/checkpoints'

os.makedirs(DATA, exist_ok=True)
os.makedirs(CKPT, exist_ok=True)

print(f'📁 Work: {WORK}')
print(f'📁 Data: {DATA}')
print(f'📁 Checkpoints: {CKPT}')

In [ ]:
#@title 3️⃣ Clone Repository
import os

if not os.path.exists(REPO):
    !git clone https://github.com/mouuuuaad/MouaadNet-Ultra.git {REPO}
else:
    !cd {REPO} && git pull

os.chdir(REPO)
print(f'✅ Repository ready: {os.getcwd()}')

In [ ]:
#@title 4️⃣ Download COCO Dataset
import os

def download(url, path):
    if not os.path.exists(path):
        print(f'📥 Downloading {url.split("/")[-1]}...')
        !wget -q --show-progress {url} -O {path}.zip
        !cd {DATA} && unzip -q {path}.zip && rm {path}.zip
    else:
        print(f'✅ Exists: {path}')

download('http://images.cocodataset.org/zips/train2017.zip', f'{DATA}/train2017')
download('http://images.cocodataset.org/zips/val2017.zip', f'{DATA}/val2017')
download('http://images.cocodataset.org/annotations/annotations_trainval2017.zip', f'{DATA}/annotations')

print('\n✅ COCO Dataset Ready!')
!ls {DATA}

In [ ]:
#@title 5️⃣ Run Training
EPOCHS = 50  #@param {type:"integer"}
BATCH_SIZE = 16  #@param {type:"integer"}
LR = 0.001  #@param {type:"number"}

!python training/train_detection.py \
    --data {DATA} \
    --epochs {EPOCHS} \
    --batch-size {BATCH_SIZE} \
    --lr {LR} \
    --save-dir {CKPT} \
    --export

In [ ]:
#@title 6️⃣ Visualize Results
import matplotlib.pyplot as plt
import torch

# Load history from checkpoint
ckpt = torch.load(f'{CKPT}/best.pt', map_location='cpu')
history = ckpt.get('history', {})

if history:
    plt.figure(figsize=(10, 4))
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'], label='Val')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Training (Best: {ckpt["best_loss"]:.4f})')
    plt.grid(True)
    plt.savefig(f'{CKPT}/curves.png')
    plt.show()

print(f'\n✅ Best loss: {ckpt["best_loss"]:.4f}')
print(f'📁 Files: {CKPT}')

In [ ]:
#@title 7️⃣ Test Detection
import sys
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
from PIL import Image

sys.path.insert(0, REPO)
from mouaadnet_ultra.model import MouaadNetUltra

# Load model
model = MouaadNetUltra()
ckpt = torch.load(f'{CKPT}/best.pt', map_location='cpu')
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

# Test on sample image
img_path = f'{DATA}/val2017/000000001268.jpg'  # Sample
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = cv2.resize(img, (416, 416))

# Preprocess
x = img_resized.astype(np.float32) / 255.0
x = (x - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
x = torch.from_numpy(x.transpose(2, 0, 1)).unsqueeze(0).float()

# Inference
with torch.no_grad():
    out = model(x)
    hm = torch.sigmoid(out['heatmaps'][0][0, 0]).numpy()

# Show
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.imshow(img_resized)
ax1.set_title('Input')
ax1.axis('off')
ax2.imshow(hm, cmap='hot')
ax2.set_title(f'Heatmap (max={hm.max():.2f})')
ax2.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
#@title 8️⃣ Download Models
import os

print('📦 Available models:')
for f in os.listdir(CKPT):
    if f.endswith('.pt') or f.endswith('.onnx'):
        size = os.path.getsize(f'{CKPT}/{f}') / 1e6
        print(f'   {f} ({size:.1f} MB)')

# Download (Colab only)
try:
    from google.colab import files
    files.download(f'{CKPT}/best.pt')
    files.download(f'{CKPT}/detection.onnx')
except:
    print(f'\n📁 Copy from: {CKPT}')

print('\n🎉 Use with webcam:')
print('python examples/webcam_demo.py --weights best.pt')